# Data cleaning Final project - CFF Railway and Bus Network

This notebook contains all the data cleaning done for the final project. Note that the running times are very long!

In [1]:
import pandas as pd 
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle 

# To hide the warnings
import warnings
warnings.filterwarnings('ignore')

## Identification of the relevant routes, trips and stops

In this first part, we decided to clean and select only the routes, trips and stops corresponding to buses and train. We first select the route_id corresponding to relevant type then we extract the relevant trip_id corresponding to these routes and finally we extract the stops contained in this trips.
All these data will be save in .csv file.

In [3]:
# Import the data for the routes, trips and stops from the .txt file.
# In the repository stop_times.txt and trips.txt have to be extracted for the ZIP files because of the files' size. 
data_routes = pd.read_csv('routes.txt', sep=",")
data_trips = pd.read_csv('trips.txt', sep=",")
data_stop = pd.read_csv('stop_times.txt', sep=",")

routes = dict()
trips = dict()
stop_id = list()
stop = dict()
valid_type = ['Eurocity','Extrazug','ICE','Intercity','InterRegio','Railjet','RegioExpress','Regionalzug','S-Bahn','Schnellzug','TGV','Bus','ICB']

# To consider only the routes relative to trains and buses
for i in range(len(data_routes)):
    if((data_routes['route_desc'][i] in valid_type) == True):
            routes[data_routes['route_id'][i]] = data_routes['route_desc'][i]
            
# To consider only trips corresponding to train and buses routes
for i in range(len(data_trips)):
    if ((data_trips['route_id'][i] in list(routes)) == True):
            trips[data_trips['trip_id'][i]] = data_trips['trip_short_name'][i]
            
# To consider only stops associated to valid trips and add latitude and longitude
for i in range(len(data_stop)):
    if((data_stop['trip_id'][i] in list(trips)) == True):
            if((data_stop['stop_id'][i] in stop_id) == False):
                stop_id.append(data_stop['stop_id'][i])

In [4]:
# Creation of the new DataFrame
data_stop_relevant = pd.DataFrame(stop_id)

data_trips_relevant = pd.DataFrame([list(trips.keys()),list(trips.values())])
data_trips_relevant = data_trips_relevant.transpose()

data_routes_relevant = pd.DataFrame([list(routes.keys()),list(routes.values())])
data_routes_relevant = data_routes_relevant.transpose()

In [9]:
# Exportation to .csv to stock the data 
data_stop_relevant.to_csv('stop_id_relevant_final.csv',sep=',')
data_trips_relevant.to_csv('trips_relevant_final.csv',sep=',')
data_routes_relevant.to_csv('routes_relevant_final.csv',sep=',')

## Proper labeling of dataframes

In this part, we import the csv created in the previous part to save running time and we reindex them. Therefore we construct data_stop_relevant_final which present all the information on the stops (name, lat, lon) indexed by the stop_id.

In [ ]:
# To import the .csv created in the first data cleaning part. 
data_stop_relevant_final = pd.read_csv('stop_id_relevant_final.csv',sep=',')
del data_stop_relevant_final["Unnamed: 0"]

data_trips_relevant_final = pd.read_csv('trips_relevant_final.csv',sep=',')
del data_trips_relevant_final["Unnamed: 0"]

data_routes_relevant_final = pd.read_csv('routes_relevant_final.csv',sep=',')
del data_routes_relevant_final["Unnamed: 0"]

In [13]:
# This cell reindex the stops DataFrame in order to be more efficient for the cleaning after
stops = pd.read_csv('stops.txt',sep=',')

del stops['location_type']
del stops['parent_station']

reindexed_stops = stops.set_index(['stop_id'])

In [ ]:
# This cell update the data_stop_relevant DataFrame by adding the name, lat and lon of the stop

index_stop = [i for i in range(len(data_stop_relevant_final))]
print(len(index_stop))
data_stop_relevant_final['stop_name'] = pd.Series([0.0 for i in range(len(data_stop_relevant_final))], index=index_stop)
data_stop_relevant_final['stop_lat'] = pd.Series([0.0 for i in range(len(data_stop_relevant_final))], index=index_stop)
data_stop_relevant_final['stop_lon'] = pd.Series([0.0 for i in range(len(data_stop_relevant_final))], index=index_stop)

data_stop_relevant_final.columns = ['stop_id','stop_name','stop_lat','stop_lon']

for i in range(len(data_stop_relevant_final)):
    if (i%2000 == 0):
        print(i)
    s = data_stop_relevant_final['stop_id'][i]
    row = reindexed_stops.loc[s]

    data_stop_relevant_final['stop_name'][i] = row['stop_name']
    data_stop_relevant_final['stop_lat'][i] = float(row['stop_lat'])
    data_stop_relevant_final['stop_lon'][i] = float(row['stop_lon'])

In [3]:
# Display the routes relevant
data_routes_relevant_final

,0,1
0,24-64-j18-1,Bus
1,24-65-j18-1,Bus
2,24-66-j18-1,Bus
3,24-67-j18-1,Bus
4,11-83-j18-1,Bus
5,1-1-A-j18-1,S-Bahn
6,1-1-B-j18-1,Intercity
7,1-1-C-j18-1,S-Bahn
8,1-1-D-j18-1,Bus
9,1-1-E-j18-1,Bus


In [18]:
# Export all the important data on stops (id, name, lat and lon)
data_stop_relevant_final.to_csv('stop_data_complete.csv',sep=',')

In [ ]:
# To label the DataFrame
data_routes_relevant_final.columns = ['route_id','route_type']
data_routes_relevant_final = data_routes_relevant_final.set_index(['route_id'])

In [5]:
# To label the DataFrame
data_trips_relevant_final.columns = ['trip_id','trip_short_name']
del data_trips_relevant_final['trip_short_name']

In [ ]:
# Cleaning trips file
trips = pd.read_csv('trips.txt',sep=',')
del trips['service_id']
del trips['trip_headsign']
del trips['trip_short_name']
del trips['direction_id']
reindexed_trips = trips.set_index(['trip_id'])


index_trips = [i for i in range(len(data_trips_relevant_final))] 
data_trips_relevant_final['route_type'] = pd.Series([0.0 for i in range(len(data_trips_relevant_final))], index=index_trips)

for i in range(len(data_trips_relevant_final)):
    if (i%10000 == 0):
        print(i)
    
    trip = data_trips_relevant_final['trip_id'][i]
    row = reindexed_trips.loc[trip]
    route = row['route_id']
    data_trips_relevant_final['route_type'][i] = str(data_routes_relevant_final['route_type'][route])    

In [8]:
# Display the routes relevant
data_routes_relevant_final

,route_type
route_id,
24-64-j18-1,Bus
24-65-j18-1,Bus
24-66-j18-1,Bus
24-67-j18-1,Bus
11-83-j18-1,Bus
1-1-A-j18-1,S-Bahn
1-1-B-j18-1,Intercity
1-1-C-j18-1,S-Bahn
1-1-D-j18-1,Bus


In [9]:
# Export the data on trips with type of transport info
data_trips_relevant_final.to_csv('trips_with_type_of_transport.csv',sep=',')

## Identification of trips sequence

In this part of the preparation, we identify the sequence of stops for each trip. In order to do that, we analyzed the "stop_times" file and extract the sequence of stops for each trip.

In [4]:
# Import the stops and trips dataframe
data_stop_relevant_final = pd.read_csv('stop_data_complete.csv',sep=',', encoding = 'latin-1')
del data_stop_relevant_final["Unnamed: 0"]

data_trips_relevant_final = pd.read_csv('trips_with_type_of_transport.csv',sep=',')
del data_trips_relevant_final["Unnamed: 0"]

In [6]:
# Import the stops times
data_stop_final = pd.read_csv('stop_times.txt', sep=",")

del data_stop_final['arrival_time']
del data_stop_final['departure_time']
del data_stop_final['pickup_type']
del data_stop_final['drop_off_type']

In [7]:
# Create a reindexation of the stops for convenience
stops = pd.read_csv('stops.txt',sep=',')

del stops['location_type']
del stops['parent_station']

reindexed_stops = stops.set_index(['stop_id'])

In [ ]:
# This cell creates The dataframe which contain all the sequence of the stops in each trip
L = list(data_trips_relevant_final['trip_id'])
List_trip_id = []
List_stop_id = []
List_stop_name = []
List_stop_seq = []

for i in range(len(data_stop_final)):
    if (i%100000 == 0):
        print(i)
    
    row = data_stop_final.loc[[i]]
    trip = row['trip_id'][i]
    stop = str(row['stop_id'][i])
    seq = row['stop_sequence'][i]
    name = reindexed_stops['stop_name'][stop] 
    
    if((trip in L)==True):
        List_trip_id.append(trip)
        List_stop_id.append(stop)
        List_stop_name.append(name)
        List_stop_seq.append(seq)
                
data_stop_sequence_detail_final = pd.DataFrame({'trip_id': List_trip_id,'stop_id': List_stop_id,'stop_name': List_stop_name,'stop_sequence': List_stop_seq})

In [138]:
# Display the stop sequence
data_stop_sequence_detail_final

,stop_id,stop_name,stop_sequence,trip_id
0,8500010:0:3,Basel SBB,1,1.TA.1-1-A-j18-1.1.R
1,8500020:0:3,Muttenz,2,1.TA.1-1-A-j18-1.1.R
2,8500021:0:5,Pratteln,3,1.TA.1-1-A-j18-1.1.R
3,8517131:0:2,Pratteln Salina Raurica,4,1.TA.1-1-A-j18-1.1.R
4,8500300:0:5,Kaiseraugst,5,1.TA.1-1-A-j18-1.1.R
5,8500313:0:2,Rheinfelden Augarten,6,1.TA.1-1-A-j18-1.1.R
6,8500301:0:3,Rheinfelden,7,1.TA.1-1-A-j18-1.1.R
7,8500302:0:3,Möhlin,8,1.TA.1-1-A-j18-1.1.R
8,8500303:0:2,Mumpf,9,1.TA.1-1-A-j18-1.1.R
9,8500320:0:3,Stein-Säckingen,10,1.TA.1-1-A-j18-1.1.R


In [137]:
# Save the stop sequence details
data_stop_sequence_detail_final.to_csv('final_stop_sequence.csv',sep=',')

## Identification of start and stop for each trip

This part extract the first and last stop of each trip. This cleaning was done in order to remove bus trips inside a city

In [150]:
# Select and identify the first and last stop of each trip
i = 0
List_trip = []
List_start = []
List_end = []

while (i<2568568):
    trip = data_stop_sequence_detail_final['trip_id'][i]
    List_trip.append(trip)
    List_start.append(data_stop_sequence_detail_final['stop_name'][i])
    while(data_stop_sequence_detail_final['trip_id'][i] == trip):
        if(i==2568567):
            i = i+1
            break
        
        i=i+1
    List_end.append(data_stop_sequence_detail_final['stop_name'][i-1])

In [153]:
data_start_end_trips = pd.DataFrame({'trip_id': List_trip,'stop_start': List_start,'stop_end': List_end})

In [158]:
# Reorder the columns
cols = ['trip_id','stop_start','stop_end']
data_start_end_trips = data_start_end_trips[cols]
data_start_end_trips

,trip_id,stop_start,stop_end
0,1.TA.1-1-A-j18-1.1.R,Basel SBB,Frick
1,5.TA.1-1-A-j18-1.1.R,Basel SBB,Frick
2,19.TA.1-1-A-j18-1.1.R,Basel SBB,Frick
3,23.TA.1-1-A-j18-1.1.R,Basel SBB,Frick
4,2.TA.1-1-A-j18-1.2.R,Basel SBB,Frick
5,3.TA.1-1-A-j18-1.3.R,Basel SBB,Frick
6,6.TA.1-1-A-j18-1.3.R,Basel SBB,Frick
7,7.TA.1-1-A-j18-1.3.R,Basel SBB,Frick
8,8.TA.1-1-A-j18-1.3.R,Basel SBB,Frick
9,9.TA.1-1-A-j18-1.3.R,Basel SBB,Frick


In [159]:
data_start_end_trips.to_csv('start_end_for_trips.csv',sep=',')

## Network generation

In this part, we will generate the network. In order to do that, we first create all the nodes from the data_stop_relevant_final created before. Then we create the edges with the data_stop_sequence_detail_final by not taking into account the buses with start and last stop in the same city.

In [10]:
data_stop_sequence_detail_final = pd.read_csv('final_stop_sequence.csv',sep=',',encoding = 'latin-1')
del data_stop_sequence_detail_final["Unnamed: 0"]

In [13]:
# To create the graph 
G = nx.Graph()

# To create the nodes (the if/else structure only forbid to create several nodes with the same name)
for i in range(len(data_stop_relevant_final)):
    name = data_stop_relevant_final['stop_name'][i]
    num_id = data_stop_relevant_final['stop_id'][i]
    lat = data_stop_relevant_final['stop_lat'][i]
    lon = data_stop_relevant_final['stop_lon'][i]

    if (name in G.nodes()) == True:
        G.node[name]['ids'].append(num_id)
    else:
        G.add_node(name,ids=[num_id])
        G.node[name]['lat'] = lat
        G.node[name]['lon'] = lon

In [15]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 21202
Number of edges: 0
Average degree:   0.0000


In [39]:
with open('stops_too_close.pkl', 'rb') as f:
    stops_too_close = pickle.load(f)

In [40]:
stops_too_close_flat = [y for x in stops_too_close.values() for y in x]

In [ ]:
# To create the edges

List_sequence_trip = []
trip = data_stop_sequence_detail_final['trip_id'][0]

for index, row in data_stop_sequence_detail_final.iterrows():
    if (index%10000==0):
        print(index)
        
    if (trip == row['trip_id']):
        List_sequence_trip.append(row['stop_name'])
    
    if ((trip != row['trip_id']) or (index == len(data_stop_sequence_detail_final))):
        start = List_sequence_trip[0]
        end = List_sequence_trip[-1]
    

        if not ((start in stops_too_close_flat) and (end in stops_too_close_flat)):
            for j in range((len(List_sequence_trip) - 1)):
                if (List_sequence_trip[j],List_sequence_trip[j+1]) in G.edges():
                    G.edge[List_sequence_trip[j]][List_sequence_trip[j+1]]['ids'].append(trip)
                else:
                    G.add_edge(List_sequence_trip[j],List_sequence_trip[j+1], ids=[trip])
        
        trip = row['trip_id']
        List_sequence_trip = [row['stop_name']]  
    

In [ ]:
# Print the basic information about the network generated
print(nx.info(G))

In [161]:
# Save the network generated in the .gml standard file
nx.write_gml(G,'CFF_Graph_Final.gml')

## Add node type

In this part we add the node type (train or bus) inside the network.

In [26]:
# Get the type of the node 
reindexed_data_trips_relevant_final = data_trips_relevant_final.set_index(['trip_id'])
attributes = nx.get_edge_attributes(G,'ids')
dict_type = dict()
for e in G.edges():
    trip = str(attributes[e])
    type_trip = reindexed_data_trips_relevant_final['route_type'][trip]
    stop1 = e[0]
    stop2 = e[1]
    
    if (type_trip == 'ICB'):
        type_trip = 'Bus'
    if (type_trip != 'Bus'):
        type_trip = 'Train'
        
    if ((stop1 in dict_type.keys()) == False):
        dict_type[stop1] = type_trip
    if ((stop2 in dict_type.keys()) == False):
        dict_type[stop2] = type_trip

In [267]:
# Add the attribute for all nodes
nx.set_node_attributes(G, 'NodeType', dict_type)

In [29]:
# To assign the not known type
A = dict_type.keys()
B = G.nodes()
Diff = list(set(A).symmetric_difference(set(B)))
for name in Diff:
    dict_type[name] = 'Train'
    
nx.set_node_attributes(G, 'NodeType', dict_type)

In [33]:
stop_with_type = pd.DataFrame({'stop': list(dict_type.keys()),'type': list(dict_type.values())})
stop_with_type.to_csv('stop_with_type.csv')

## Add population to node

This part add the attribute population to all the nodes.

In [ ]:
G = nx.read_gml('CFF_Graph_Final_2.gml')

In [ ]:
data_stops_with_population = pd.read_csv('stops_with_population.csv')

In [ ]:
for node in G.nodes():
    pop_values = data_stops_with_population.loc[data_stops_with_population['stop'] == node]['population'].values
    if len(pop_values) > 0:
        population = pop_values[0]
    else:
        population = None
    G.node[node]

Now each stop in the data has been represented as one node in the network. A node has atleast a latitude and longitude and an ID. If multiple IDs where present (for example for multiple platforms in a train station), then they are stored in a list in the node attribute. If a population was found for the stop, then it also has a population, otherwise the population attribute is None.

In [23]:
# Save the network generated in the .gml standard file
nx.write_gml(G,'CFF_Graph_Final_2.gml')